In [4]:
import sys
sys.path.append('../')

In [5]:
%set_env PYTHONPATH ../

env: PYTHONPATH=../


In [6]:
import ltorch
from ltorch.nn import TransformerLayer

# 定义模型参数
d_model = 512  # 输入和输出特征的维度
nhead = 8      # 多头注意力机制中的头数
dim_feedforward = 2048  # 前馈神经网络的隐藏层维度
dropout = 0.0  # Dropout概率

ldevice = ltorch.cuda()
ltorch_model = TransformerLayer(d_model, nhead, d_model//nhead, dim_feedforward, dropout=dropout, device=ldevice, dtype="float32")

Using ltorch backend


In [7]:
import torch
from torch.nn import TransformerEncoderLayer
device = torch.device("cuda")
torch_model = TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, batch_first=True).to(device)
torch_model.linear1.bias.data.zero_()
torch_model.linear2.bias.data.zero_()
M = torch.triu(-float("inf")*torch.ones(10, 10),1).to(device)

In [8]:
import numpy as np
# 随机生成输入数据
batch_size = 64
seq_len = 10
input_size = 512
inputs = np.random.rand(batch_size, seq_len, input_size).astype(np.float32)

# 转换为torch tensor
inputs_torch = torch.tensor(inputs, device=device)

# 转换为ltorch tensor
inputs_ltorch = ltorch.Tensor(inputs, device=ldevice, dtype="float32", requires_grad=False)

In [9]:
outputs_torch = torch_model(inputs_torch, M)

In [10]:
outputs_torch.shape

torch.Size([64, 10, 512])

In [11]:
outputs_ltorch = ltorch_model(inputs_ltorch)

In [12]:
outputs_ltorch.shape

(64, 10, 512)

In [13]:
ltorch_model.attention.k_projection.weight.shape

(512, 512)

In [14]:
ltorch_model.attention.out_projection.weight.shape

(512, 512)

In [15]:
torch_model.self_attn.in_proj_weight[:512].shape

torch.Size([512, 512])

In [16]:
torch_model.self_attn.out_proj.weight.shape

torch.Size([512, 512])

In [17]:
import ltorch
from ltorch.apps.models import LanguageModel
from ltorch.apps.simple_ml import train_ptb, evaluate_ptb

device = ltorch.cuda()
corpus = ltorch.data.Corpus("./ltorch_data/ptb")
train_data = ltorch.data.batchify(corpus.train, batch_size=64, device=device, dtype="float32")
model = LanguageModel(20, len(corpus.dictionary), hidden_size=32, num_layers=12, seq_model='transformer', seq_len=20, device=device)
train_ptb(model, train_data, seq_len=20, n_epochs=10, device=device, lr=0.003, optimizer=ltorch.optim.Adam)
evaluate_ptb(model, train_data, seq_len=20, device=device)

100%|██████████| 727/727 [12:57<00:00,  1.07s/it]


avg_acc: 0.044006704881911454, avg_loss: 7.291937828063965


100%|██████████| 727/727 [12:57<00:00,  1.07s/it]


avg_acc: 0.05124414721476279, avg_loss: 6.597293376922607


100%|██████████| 727/727 [12:56<00:00,  1.07s/it]


avg_acc: 0.05165298147765613, avg_loss: 6.587510585784912


100%|██████████| 727/727 [12:54<00:00,  1.06s/it]


avg_acc: 0.05151096536528266, avg_loss: 6.580854415893555


 49%|████▊     | 354/727 [06:20<06:41,  1.08s/it]


KeyboardInterrupt: 